In [1]:
import pandas as pd
import os
import geopandas as gpd
from keplergl import KeplerGl
import json


In [2]:
morbi = pd.read_csv('../data/morbilidad_Fgen.csv')

In [3]:
morbi.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
296388,28,2,28,2,F329,1,60,11,1.01195,F32,...,2018-07-04,2018-06-23,2018,6,23,Urgente,Curación,Mujer,Madrid,Madrid
142747,28,1,28,1,F200,1,49,30,1.00000,F20,...,2017-11-08,2017-10-09,2017,10,9,Ordinario,Curación,Hombre,Madrid,Madrid
288341,30,2,30,2,F3160,1,42,20,1.00000,F31,...,2018-01-11,2017-12-22,2017,12,22,Urgente,Curación,Mujer,Murcia,Murcia


In [4]:
morbi.fecha_ing =  pd.to_datetime(morbi.fecha_ing)
morbi.fecha_al =  pd.to_datetime(morbi.fecha_al)

In [7]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [45]:
# #Create a basemap 
# map_0= KeplerGl(height=700, weight = 500, data={'morbilidad': morbi})
# #show the map
# map_0

# map_0.add_data(data=provincias, name='geojson')

In [6]:
total_pro = morbi.groupby('ProvHosp_nom').agg({'SexNom':'count'}).reset_index().rename({'SexNom':'ingresos'},axis=1)

In [7]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [8]:
poly = {p['properties']['name']:p['geometry'] for p in provincias['features'] }

In [9]:
total_pro['pol'] = total_pro['ProvHosp_nom'].map(poly)

In [10]:
total_pro.sample()

,ProvHosp_nom,ingresos,pol
10,Castellón,2858,"{'type': 'MultiPolygon', 'coordinates': [[[[-0..."


In [11]:
total_pob = pd.read_csv('../data/poblacion_prov.csv')

In [12]:
total_pob = total_pob[total_pob['Sexo'] == 'Total']

In [13]:
total_pob.sample(3)

,Provincias,cod_pr,Sexo,Total
134,Toledo,45,Total,633374.67
83,Madrid,28,Total,6059572.96
44,A Coruña,15,Total,1125365.04


In [14]:
dic_pob = {}
for i,r in total_pob.iterrows():
    dic_pob[r.Provincias] = r.Total

In [15]:
total_pro['poblacion'] = total_pro.ProvHosp_nom.map(dic_pob)

In [16]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion
14,Cáceres,5788,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",408155.50
47,Vizcaya,17638,"{'type': 'MultiPolygon', 'coordinates': [[[[-2...",1144973.25
13,Cuenca,1327,"{'type': 'MultiPolygon', 'coordinates': [[[[-2...",205901.75


In [17]:
total_pro['ponde'] = ((total_pro.ingresos/total_pro.poblacion)*100000).round(2)

In [18]:
total_pro.sample(3)

,ProvHosp_nom,ingresos,pol,poblacion,ponde
51,Ávila,1215,"{'type': 'MultiPolygon', 'coordinates': [[[[-5...",166206.83,731.02
26,León,3568,"{'type': 'MultiPolygon', 'coordinates': [[[[-4...",490686.42,727.14
19,Guadalajara,1535,"{'type': 'MultiPolygon', 'coordinates': [[[[-2...",221341.29,693.50


In [21]:
total_pro.to_csv('../output/kp_pro_tot.csv',index=False)

In [20]:
###### Create a basemap 
map_1= KeplerGl(height=700, weight = 500, data={'total_pro': total_pro})
#show the map
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'total_pro':               ProvHosp_nom  ingresos  \
0                 A Coruña      7899   
1 …

In [62]:
morbi.sample()

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
78113,28,1,28,2,F333,1,69,13,1.08548,F33,...,2016-05-10,2016-04-27,2016,4,27,Urgente,Curación,Hombre,Madrid,Madrid


In [63]:
pro_day = morbi.groupby(['ProvHosp_nom','year','month']).agg({'day':'count','fecha_ing':'min'}).reset_index().rename({'day':'total'},axis=1)

In [64]:
pro_day.sample()

,ProvHosp_nom,year,month,total,fecha_ing
477,Baleares,2014,12,5,2014-12-03


In [65]:
pro_day.shape

(3997, 5)

In [66]:
pro_day['geometry']= pro_day.ProvHosp_nom.map(poly)

In [ ]:
map_2= KeplerGl(height=700, weight = 700, data={'Ingresos_dia': pro_day})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
